In [ ]:
from utils import *
import carla
from detectors import torch_script_module
from carla_control import carla_client
# from utils.mods.post_processing import mono_label_distance_tracker, multi_classes_assemble_tracker
import numpy as np
import open3d as o3d
import torch
import time

try:
    from pcdet.config import cfg, cfg_from_yaml_file
    from pcdet.datasets import DatasetTemplate
    from pcdet.models import build_network
    from pcdet.utils import common_utils
except Exception as e:
    print(e)
    print("pcdet is not installed")


from IPython.display import clear_output, display


In [ ]:
logger = create_logger()

In [ ]:
client = carla_client()

In [ ]:
client.init_client()
client.start_client()

In [ ]:
client.debug_luanch_test_2(24)

CFG_FILE = ".\\configs\\carla_predict_eval.yaml"
cfg = cfg_from_yaml_file(CFG_FILE, cfg)

pcs_dataset = carla_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, lidar=client.lidar_t, class_names=cfg.CLASS_NAMES)
# object_tracker = multi_classes_assemble_tracker(num_classes=4, track_length=25, multi_head=True)
try:
    model = torch.jit.load("./torch_scripts/point_pillar_model.pt")
    model.cuda()
    model.eval()
except Exception as e:
    logger.error(f"Failed to load jit model from: {e}")
    model = None

try:
    if model is None:
        # raise NotImplementedError
        cfg_from_yaml_file("cfgs/kitti_models/pointpillar.yaml", cfg)
        model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=pcs_dataset)
        model.load_params_from_file(filename="D:/modelzoo/pointpillar_7728.pth", logger=logger, to_cpu=True)
        model.cuda()
        model.eval()
except Exception as e:
    print(e)
    logger.error("Failed to load model from D:/modelzoo/pointpillar_7728.pth")
print(cfg)

In [ ]:
def carla_ticking(idx, data_dict): 
    before_time = time.perf_counter()
    
    client.carla_world.tick()
    # ego_vehicle_transform = client.vehicle.get_transform()
    # ego_vehicle_location = ego_vehicle_transform.location
    # ego_vehicle_rotation = ego_vehicle_transform.rotation
    # offset = carla.Vector3D(x=-8.5, y=0, z=3.5)
    # left_rear_location = ego_vehicle_location + ego_vehicle_rotation.get_right_vector() * offset.y + \
    #                                         ego_vehicle_rotation.get_forward_vector() * offset.x + \
    #                                         ego_vehicle_rotation.get_up_vector() * offset.z

    # spectator = client.carla_world.get_spectator()
    # spectator_transform = carla.Transform(left_rear_location, ego_vehicle_rotation)
    # spectator.set_transform(spectator_transform)
        
    after_time = time.perf_counter()
    return after_time - before_time, None
    
def model_forwarding(idx, data_dict):
    final_boxes, final_scores, final_labels = None, None, None
    before_time = time.perf_counter()
    
    if data_dict["points"] is not None:
        pred_dicts, _ = model.forward(data_dict)

        if isinstance(pred_dicts, list):
            cls_preds = pred_dicts[0]["pred_scores"]
            box_preds = pred_dicts[0]["pred_boxes"]
            label_preds = pred_dicts[0]["pred_labels"]
        else:
            cls_preds = pred_dicts["pred_scores"]
            box_preds = pred_dicts["pred_boxes"]
            label_preds = pred_dicts["pred_labels"]
            selected, selected_scores = class_agnostic_nms(
                                box_scores=cls_preds, box_preds=box_preds,
                                score_thresh=0.4
                            )
            cls_preds = cls_preds[selected]
            label_preds = label_preds[selected]
            box_preds = box_preds[selected]

        final_scores = cls_preds
        final_labels = label_preds
        final_boxes = box_preds
    after_time = time.perf_counter()
    return after_time - before_time, data_dict, final_boxes, final_scores, final_labels

def scene_rendering(idx, points, vis, final_boxes=None, final_scores=None, final_labels=None, gt_boxes=None):
    before_time = time.perf_counter()
    if points is not None:
        draw_scenes(vis,
                    points=points[:, 1:], ref_boxes=final_boxes,
                    ref_scores=final_scores, ref_labels=final_labels, gt_boxes=gt_boxes, confidence=None
                )
    
    vis.poll_events()
    vis.update_renderer()
    vis.clear_geometries()
    after_time = time.perf_counter()
    return after_time - before_time, None

In [ ]:
visualizers = {}

for channel_name in pcs_dataset.preview_channel:
    visualizers[channel_name] = o3d.visualization.Visualizer()
    visualizers[channel_name].create_window(window_name=channel_name)
try:
    with torch.no_grad():
        for idx, data_series in enumerate(pcs_dataset):
            
            ticking_time, _ = carla_ticking(idx, None)
            
            for channel_name, channel in data_series.items():
                data_dict = data_series[channel_name]
                vis = visualizers[channel_name]
                
                pre_time = data_dict["pre_time"]
                forward_time = 0.0
                final_boxes, final_scores, final_labels = None, None, None
                gt_boxes = data_dict.get("gt_boxes", None)
                
                if data_dict["points"] is not None:
                    
                    data_dict = pcs_dataset.collate_batch([data_dict])
                    load_data_to_gpu(data_dict)

                    if model is not None:
                        forward_time, data_dict, final_boxes, final_scores, final_labels = model_forwarding(idx, data_dict)
                    else:
                        forward_time = 0.0
                        final_boxes = None 
                        final_scores = None
                        final_labels = None

                    render_time, _ = scene_rendering(idx, data_dict["points"], vis, final_boxes, final_scores, final_labels, None)
                    
                    # render_time = time.perf_counter()
                    
                    # if final_boxes is not None:
                    #     object_tracker.updates_object(final_boxes[np.newaxis, :, :], final_labels[np.newaxis, :], final_scores[np.newaxis, :])

                    # bounding_boxes, box_scores, box_labels, tracks = object_tracker.get_all_object()

                    # draw_scenes(vis,
                    #     points=data_dict['points'][:, 1:], ref_boxes=bounding_boxes,
                    #     ref_scores=box_scores, ref_labels=box_labels, confidence=None, tracks=tracks
                    # )
                    # draw_scenes(vis,
                    #     points=data_dict['points'][:, 1:], ref_boxes=None,
                    #     ref_scores=None, ref_labels=None, confidence=None, tracks=None
                    # )
                    # vis.poll_events()
                    # vis.update_renderer()
                    # vis.clear_geometries()
                    
                    # render_time = time.perf_counter() - render_time
                    
                    clear_output(wait=True)
                    logger.info(f"Channel name: {channel_name}")
                    logger.info(f"Compute time: {pre_time:.3f} + {ticking_time:.3f} + {forward_time:.3f} + {render_time:.3f} == {pre_time + ticking_time + forward_time + render_time:.3f}s")
                    logger.info(f"Target amount: {len(final_boxes if (final_boxes is not None) else [])}")
                    # logger.info(f"current uuid:{object_tracker.get_last_uuid()}")
except KeyboardInterrupt:
    pass


(124788, 4)
(57300, 4)
BoundingBox(Location(x=0.000000, y=0.000342, z=0.781807), Extent(x=1.852685, y=0.894339, z=0.774525), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))

{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}